## Feature Creation

1. Time 
2. For each customer
3. For each terminal

In [22]:
import os
import datetime

In [1]:
# !curl -O https://raw.githubusercontent.com/Fraud-Detection-Handbook/fraud-detection-handbook/main/Chapter_References/shared_functions.py
# %run shared_functions.py
from shared_functions import read_from_files

##### Load data

In [2]:
DIR_INPUT = '../data/'
BEGIN_DATE = '2018-04-01'
END_DATE = '2018-09-30'

print('Loading files') 
transactions_df = read_from_files(DIR_INPUT, 'simulated_fraud_data_', BEGIN_DATE, END_DATE)
print(f"{len(transactions_df)} transactions loaded, containing {transactions_df.TX_FRAUD.sum()} fraudulent transactions")

Loading files
1754155 transactions loaded, containing 14681 fraudulent transactions


##### Create "during weekend?" and "during night?" features

In [3]:
transactions_df['TX_DURING_WEEKEND'] = transactions_df.TX_DATETIME.apply(lambda datetime: int(datetime.weekday() >= 5))
transactions_df['TX_DURING_NIGHT'] = transactions_df.TX_DATETIME.apply(lambda datetime: int(datetime.hour <= 6))

In [4]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,1,1
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,1,1
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0,1,1
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0,1,1
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0,1,1


##### Customer transformations

Frequency: the number of transactions that occured over the specified windows (1, 7, and 30 days).

Monetary: the average value of transactions that occured over the specified windows (1, 7, and 30 days).

Results in 6 new features.

In [5]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1, 7, 30]):
    customer_transactions = customer_transactions.sort_values('TX_DATETIME')
    
    customer_transactions.index = customer_transactions.TX_DATETIME # this lets us use days in the rolling window
    
    for window in windows_size_in_days:
        # frequency feature
        num_tx_window = customer_transactions.TX_AMOUNT.rolling(str(window) + 'd').count()
        # monetary feature
        avg_val_window = customer_transactions.TX_AMOUNT.rolling(str(window) + 'd').sum() / num_tx_window
            
        customer_transactions['CUSTOMER_ID_NB_TX_' + str(window) + 'DAY_WINDOW'] = list(num_tx_window) 
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_' + str(window) + 'DAY_WINDOW'] = list(avg_val_window)
        
    customer_transactions.index = customer_transactions.TRANSACTION_ID
    return customer_transactions

get_customer_spending_behaviour_features(transactions_df[transactions_df.CUSTOMER_ID==0]).describe()

,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
count,3.840000e+02,384.0,384.000000,384.000000,3.840000e+02,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000
mean,9.413650e+05,0.0,4326.000000,61.448750,8.482639e+06,97.671875,0.002604,0.005208,0.315104,0.153646,3.255208,61.372495,15.145833,62.022746,56.750000,62.203857
std,5.419399e+05,0.0,3034.725054,28.787885,4.885717e+06,56.546999,0.051031,0.102062,0.465163,0.361079,1.742563,18.873249,4.857723,9.320656,15.859863,5.365408
min,1.758000e+03,0.0,29.000000,0.870000,2.634500e+04,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.300000,1.000000,41.823571,1.000000,55.503704
25%,4.376832e+05,0.0,1536.000000,39.487500,3.936543e+06,45.000000,0.000000,0.000000,0.000000,0.000000,2.000000,52.066250,12.000000,55.967708,52.000000,59.114817
50%,9.829680e+05,0.0,3383.000000,61.895000,8.855476e+06,102.000000,0.000000,0.000000,0.000000,0.000000,3.000000,61.206667,15.000000,60.782063,58.000000,61.367431
75%,1.440398e+06,0.0,6962.000000,80.605000,1.299075e+07,150.000000,0.000000,0.000000,1.000000,0.000000,4.000000,71.630000,18.000000,66.603846,65.000000,64.326751
max,1.751455e+06,0.0,9920.000000,148.160000,1.577950e+07,182.000000,1.000000,2.000000,1.000000,1.000000,10.000000,127.150000,31.000000,123.590000,91.000000,123.590000


In [6]:
transactions_customer_features_df = transactions_df.groupby('CUSTOMER_ID').apply(lambda customer:
    get_customer_spending_behaviour_features(customer, windows_size_in_days=[1, 7, 30]))

In [7]:
transactions_customer_features_df = transactions_customer_features_df.sort_values('TX_DATETIME').reset_index(drop=True)
transactions_customer_features_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,1,1,1.0,57.160000,1.0,57.160000,1.0,57.160000
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,1,1,1.0,81.510000,1.0,81.510000,1.0,81.510000
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0,1,1,1.0,146.000000,1.0,146.000000,1.0,146.000000
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0,1,1,1.0,64.490000,1.0,64.490000,1.0,64.490000
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0,1,1,1.0,50.990000,1.0,50.990000,1.0,50.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2018-09-30 23:56:36,161,655,54.24,15810996,182,0,0,1,0,2.0,75.280000,12.0,67.047500,72.0,69.521111
1754151,1754151,2018-09-30 23:57:38,4342,6181,1.23,15811058,182,0,0,1,0,1.0,1.230000,21.0,22.173810,93.0,24.780753
1754152,1754152,2018-09-30 23:58:21,618,1502,6.62,15811101,182,0,0,1,0,5.0,7.368000,21.0,7.400476,65.0,7.864462
1754153,1754153,2018-09-30 23:59:52,4056,3067,55.40,15811192,182,0,0,1,0,3.0,100.696667,16.0,107.052500,51.0,102.919608


##### Terminal transformations

In [16]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30]):
    terminal_transactions = terminal_transactions.sort_values('TX_DATETIME')
    terminal_transactions.index = terminal_transactions.TX_DATETIME # this lets us use days in the rolling window
    
    # calculate delay
    num_fraud_tx_delay = terminal_transactions.TX_FRAUD.rolling(str(delay_period) + 'd').sum()
    qty_fraud_tx_delay = terminal_transactions.TX_FRAUD.rolling(str(delay_period) + 'd').count()
    
    for window in windows_size_in_days:
        
        # calculate delay+window size
        num_fraud_tx_delay_and_window = terminal_transactions.TX_FRAUD.rolling(str(delay_period + window) + 'd').sum()
        qty_fraud_tx_delay_and_window = terminal_transactions.TX_FRAUD.rolling(str(delay_period + window) + 'd').count()
        
        # remove effect of the last delay period to create the delayed response to detecting fraudulent/risky terminals
        num_fraud_tx_window = num_fraud_tx_delay_and_window - num_fraud_tx_delay
        qty_fraud_tx_window = qty_fraud_tx_delay_and_window - qty_fraud_tx_delay
        
        # risk quantifier for the terminal in the window (between 0 and 1)
        risk_window = num_fraud_tx_window / qty_fraud_tx_window
        
        terminal_transactions['TERMINAL_ID_NB_TX_' + str(window) + 'DAY_WINDOW'] = list(num_fraud_tx_window)
        terminal_transactions['TERMINAL_ID_RISK_' + str(window) + 'DAY_WINDOW'] = list(risk_window)
        
    terminal_transactions.index = terminal_transactions.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions

In [17]:

get_count_risk_rolling_window(transactions_df[transactions_df.TERMINAL_ID==3059], delay_period=7, windows_size_in_days=[1,7,30])

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,
3527,3527,2018-04-01 10:17:43,3774,3059,225.41,37063,0,1,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0
4732,4732,2018-04-01 11:59:14,55,3059,36.28,43154,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0
16216,16216,2018-04-02 14:47:34,4879,3059,105.00,139654,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
18249,18249,2018-04-02 19:08:10,2263,3059,90.89,155290,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
26512,26512,2018-04-03 15:44:49,4879,3059,58.51,229489,2,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697944,1697944,2018-09-25 05:32:56,402,3059,57.30,15312776,177,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
1701971,1701971,2018-09-25 12:30:54,1035,3059,7.56,15337854,177,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1704512,1704512,2018-09-25 16:37:41,1519,3059,35.79,15352661,177,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
%time transactions_all_transforms = transactions_customer_features_df.groupby('TERMINAL_ID').apply(lambda terminals: get_count_risk_rolling_window(terminals, delay_period=7, windows_size_in_days=[1,7,30]))
transactions_all_transforms = transactions_all_transforms.sort_values('TX_DATETIME').reset_index(drop=True)

CPU times: user 1min 32s, sys: 0 ns, total: 1min 32s
Wall time: 1min 33s


##### Save the dataset

In [23]:
DIR_OUTPUT = "../data/simulated-transformed/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.datetime.strptime("2018-04-01", "%Y-%m-%d")

for day in range(transactions_all_transforms.TX_TIME_DAYS.max()+1):
    
    transactions_day = transactions_all_transforms[transactions_all_transforms.TX_TIME_DAYS==day].sort_values('TX_TIME_SECONDS')
    
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    # Protocol=4 required for Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)